In [1]:
!rm ../tests/fixtures/smoldyn/model_out.txt
!rm ../tests/fixtures/smoldyn/fe2_binding.simularium
!rm ../tests/fixtures/smoldyn/lip5.simularium

rm: ../tests/fixtures/smoldyn/model_out.txt: No such file or directory
rm: ../tests/fixtures/smoldyn/fe2_binding.simularium: No such file or directory
rm: ../tests/fixtures/smoldyn/lip5.simularium: No such file or directory


In [2]:
import smoldyn

fp = '/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/lip5.txt'      # lip.txt'
fp1 = '/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/fe2_binding2.txt'
fp2 = '/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/tracking.txt'
fp3 = '/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/crowding4.txt'
sbml_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Mitchell2013/BIOMD0000000498_url.xml'

model_file = fp1
fname = model_file.split('/')[-1].replace('.txt', '')

In [3]:
sim = smoldyn.Simulation.fromFile(model_file)
sim.dt

0.05

In [ ]:
sim.runSim()

In [ ]:
!ls ../tests/fixtures/smoldyn

In [2]:
from IPython.display import Image
import numpy as np
from simulariumio.smoldyn import SmoldynConverter, SmoldynData
from simulariumio import (
    UnitData, 
    MetaData, 
    DisplayData, 
    DISPLAY_TYPE, 
    ModelMetaData, 
    BinaryWriter, 
    InputFileData,
)
from simulariumio.filters import TranslateFilter


ub = 40.
box_size = 100.0
smallest_ratio = 0.5
simularium_size_ratios = {
    'LIP(solution)': smallest_ratio,  # (smallest)
    'TfR(solution)': 14.7 * smallest_ratio,
    'Tf-Fe_intercell(solution)': 11.2 * smallest_ratio,
    'Fpn(solution)': 10.3 * smallest_ratio,
    'Fe-FT(solution)': 20.3 * smallest_ratio,
    'FT(solution)': 20.3 * smallest_ratio,
    'Tf-Fe-TfR1(solution)': 13 * smallest_ratio,
}


display_data = {
    key: DisplayData(
        name=key,
        radius=1.2 if value > 15 else 1.0 if value > 8 and value < 15 else 0.5,
        display_type=DISPLAY_TYPE.SPHERE
    )
    for key, value in simularium_size_ratios.items()
}


trajectory_data = SmoldynData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="LIP",
            version="8.1",
            authors="Alexander A. Patrie",
            description=(
                "Based on BIOMD0000000498 - Mitchell, Mendes PLoS Comput. Biol. 2013 Nov; 9(11): e1003299: A computational model of liver iron metabolism"
            ),
            doi="10.1371/journal.pcbi.1003299",
            source_code_url="https://github.com/vivarium-collective/bio-bundles",
            source_code_license_url="https://github.com/vivarium-collective/bio-bundles/blob/main/LICENSE",
        ),
    ),
    smoldyn_file=InputFileData(
        file_path="/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/model_out.txt"
    ),
    # display_data=display_data,
    time_units=UnitData("s"),
    spatial_units=UnitData("µm"),  
    center=True,
)

In [3]:
converter = SmoldynConverter(trajectory_data)

simularium_fp = f"../tests/fixtures/smoldyn/{fname}"
converter.add_number_of_agents_plot()
converter.save(simularium_fp, binary=True)

Reading Smoldyn Data -------------
Reading Scatter Plot Data -------------
Converting Trajectory Data to Binary -------------
Writing Binary -------------
saved to ../tests/fixtures/smoldyn/fe2_binding.simularium


In [14]:
import json 

with open(simularium_fp + '.simularium', 'r') as f:
    trajectory = json.load(f)
    
trajectory.keys()

dict_keys(['trajectoryInfo', 'spatialData', 'plotData'])

In [9]:
for i, d in enumerate(trajectory.get('spatialData').get('bundleData')):
    # print(i, d['time'])
    pass

In [3]:
import numpy as np

def calculate_radius(m: float, rho: float, scaling_factor: float = 10**(-1)) -> float:
    """
    Calculate the radius of an agent given its molecular mass and density.

    Args:
        m (float): The molecular mass of the given agent/particle (Daltons).
        rho (float): The density of the given agent/particle (kg/m^3).
        scaling_factor (float): A scaling factor to convert radius units. Defaults to 10**(-2).

    Returns:
        float: Radius of the given agent in meters.
    """
    dalton_to_kg = 1.66053906660e-27  # Conversion factor from Daltons to kilograms
    m_kg = m * dalton_to_kg  # Convert mass to kilograms
    radius_m = ((3 * m_kg) / (4 * np.pi * rho)) ** (1 / 3)  # Calculate radius in meters
    return radius_m   # * scaling_factor  # Return radius in meters


def calculate_molecular_mass(n_amino_acids: int, mol_mass: float = None) -> float:
    """
    Calculate the molecular mass for an agent, given the amount of amino acids in the particular agent.

    Args:
        n_amino_acids (int): Number of amino acids within the given agent.
        amino_acid_mass (Optional[int]): Average molecular weight of amino acids. Defaults to 110.

    Returns:
        float: The molecular mass of the given agent (in Daltons).
    """
    avg_amino_acid_mass = 110
    if n_amino_acids > 0:
        return float(n_amino_acids * avg_amino_acid_mass)
    else:
        return mol_mass


def calculate_diffusion_coefficient(name: str, n_amino_acids: float, rho: float, T: float = 310.15, eta: float = 0.01, mol_mass: float = None) -> dict[str, float]:
    """
    Calculate the diffusion coefficient using the Stokes-Einstein equation.

    Args:
        m (float): Molecular mass (Daltons).
        rho (float): Density of the molecule (kg/m^3).
        T (float): Temperature (in Kelvin). Defaults to 310.15 (human liver cell).
        eta (float): Dynamic viscosity of the medium (Pa.s). Defaults to 0.003 (for liver cell cytoplasm)

    Returns:
        float: Diffusion coefficient (in µ^2/s).
    """
    k_B = 1.38e-23  # Boltzmann constant (in J/K)
    m = calculate_molecular_mass(n_amino_acids, mol_mass)
    # Calculate radius in meters
    radius = calculate_radius(m, rho)
    
    # Stokes-Einstein equation to calculate the diffusion coefficient
    D = (k_B * T) / (6 * np.pi * eta * radius)
    
    # Convert the diffusion coefficient to micrometers^2/s
    return D * 1e12  # Converting m^2/s to µm^2/s


species_info = {
    "Hamp": {"n_amino_acids": 25, "rho": 1350},  # Hepcidin is a small peptide (25 amino acids)
    "Fe-FT": {"n_amino_acids": 5000, "rho": 1350},  # Ferritin-bound Iron, ferritin is a large protein complex
    "FT": {"n_amino_acids": 5000, "rho": 1350},  # Ferritin alone, similar to Fe-FT
    "FT1": {"n_amino_acids": 3000, "rho": 1350},  # Smaller subunit of Ferritin
    "HO-1": {"n_amino_acids": 288, "rho": 1350},  # Heme Oxygenase-1 protein
    "Heme": {"n_amino_acids": 0, "rho": 1350, "mol_mass": 616.49},  # Heme is not a protein, but use density of small molecules
    "LIP": {"n_amino_acids": 0, "rho": 1350, "mol_mass": 55.85},  # Labile Iron Pool (assumed to be iron ions, not protein)
    "Fpn": {"n_amino_acids": 571, "rho": 1350},  # Ferroportin protein
    "IRP": {"n_amino_acids": 891, "rho": 1350},  # Iron Regulatory Protein (IRP1/2)
    "Tf-Fe_intercell": {"n_amino_acids": 679, "rho": 1350},  # Transferrin bound to iron
    "TfR": {"n_amino_acids": 760, "rho": 1350},  # Transferrin Receptor
    "Tf-Fe-TfR1": {"n_amino_acids": 1450, "rho": 1350},  # Transferrin bound to TfR1
    "HFE": {"n_amino_acids": 343, "rho": 1350},  # Hereditary Hemochromatosis Protein
    "HFE-TfR": {"n_amino_acids": 1103, "rho": 1350},  # HFE bound to Transferrin Receptor
    "Tf-Fe-TfR2": {"n_amino_acids": 1450, "rho": 1350},  # Transferrin bound to TfR2
    "2(Tf-Fe)-TfR1": {"n_amino_acids": 2900, "rho": 1350},  # Two Transferrin molecules bound to TfR1
    "2HFE-TfR": {"n_amino_acids": 2206, "rho": 1350},  # Two HFE molecules bound to TfR1
    "2HFE-TfR2": {"n_amino_acids": 2206, "rho": 1350},  # Two HFE molecules bound to TfR2
    "2(Tf-Fe)-TfR2": {"n_amino_acids": 2900, "rho": 1350},  # Two Transferrin molecules bound to TfR2
    "TfR2": {"n_amino_acids": 760, "rho": 1350},  # Transferrin Receptor 2
    "Heme_intercell": {"n_amino_acids": 0, "rho": 1350, "mol_mass": 616.49},  # Heme extracellular form
}
names = list(species_info.keys())
difcs = dict(zip(
    names,
    [calculate_diffusion_coefficient(key, **values) for key, values in species_info.items()]
))

In [4]:
difcs

{'Hamp': 24.38353181970454,
 'Fe-FT': 4.1695252906720635,
 'FT': 4.1695252906720635,
 'FT1': 4.9435188630978155,
 'HO-1': 10.796403764644122,
 'Heme': 40.13890279774164,
 'LIP': 89.37165145378914,
 'Fpn': 8.59405069383893,
 'IRP': 7.409417143706423,
 'Tf-Fe_intercell': 8.111852075785466,
 'TfR': 7.812776784688871,
 'Tf-Fe-TfR1': 6.299227302616884,
 'HFE': 10.18541136587056,
 'HFE-TfR': 6.900566991332301,
 'Tf-Fe-TfR2': 6.299227302616884,
 '2(Tf-Fe)-TfR1': 4.999700023380423,
 '2HFE-TfR': 5.476983650608965,
 '2HFE-TfR2': 5.476983650608965,
 '2(Tf-Fe)-TfR2': 4.999700023380423,
 'TfR2': 7.812776784688871,
 'Heme_intercell': 40.13890279774164}

In [10]:
from basico import * 

sbml_fp = '../tests/fixtures/sbml-core/Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations/Caravagna2010.xml'
m = load_model(sbml_fp)

In [11]:
get_reactions(model=m)

,scheme,flux,particle_flux,function,key,sbml_id,display_name,mapping
name,,,,,,,,
Induction of tumor,-> T,0.180000,1.083985e+20,Function for induction of tumor_1,Reaction_0,Induction_of_tumor,(Induction of tumor),"{'T': 'T', 'V': 'V', 'b': 'b', 'r2': 'r2'}"
Removal of tumor from the system by the action of immune response,T -> ; E,0.000003,1.881913e+15,Function for removal of the tumor_1,Reaction_1,Removal_of_tumor_from_the_system_by_the_action...,(Removal of tumor from the system by the actio...,"{'E': 'E', 'T': 'T', 'V': 'V', 'a': 'a', 'g2':..."
Activation and transfer of effector cells to the action site,-> E; I T,0.020000,1.204429e+19,Function for activation and transfer of effect...,Reaction_2,Activation_and_transfer_of_effector_cells_to_t...,(Activation and transfer of effector cells to ...,"{'E': 'E', 'I': 'I', 'T': 'T', 'V': 'V', 'c': ..."
Deactivation and removal of effector cells from the site of tumor,E ->,0.030000,1.806642e+19,Mass action (irreversible),Reaction_3,Deactivation_and_removal_of_effector_cells_fro...,(Deactivation and removal of effector cells fr...,"{'k1': 'mu2', 'substrate': 'E'}"
Activation of interleukin 2,-> I; T E,0.000488,2.939580e+17,Function for activation of interleukin 2_1,Reaction_4,Activation_of_interleukin_2,(Activation of interleukin 2),"{'E': 'E', 'T': 'T', 'V': 'V', 'g3': 'g3', 'p2..."
Deactivation of interleukin2,I ->,10.000000,6.022141e+21,Mass action (irreversible),Reaction_5,Deactivation_of_interleukin2,(Deactivation of interleukin2),"{'k1': 'mu3', 'substrate': 'I'}"


In [12]:
get_parameters(model=m)

,type,unit,initial_value,initial_expression,expression,value,rate,key,sbml_id,display_name
name,,,,,,,,,,
r2,fixed,,1.800000e-01,,,1.800000e-01,0.0,ModelValue_0,r2,Values[r2]
b,fixed,,1.000000e-09,,,1.000000e-09,0.0,ModelValue_1,b,Values[b]
a,fixed,,1.000000e+00,,,1.000000e+00,0.0,ModelValue_2,a,Values[a]
g2,fixed,,1.000000e+05,,,1.000000e+05,0.0,ModelValue_3,g2,Values[g2]
p1,fixed,,1.245000e-01,,,1.245000e-01,0.0,ModelValue_4,p1,Values[p1]
g1,fixed,,2.000000e+07,,,2.000000e+07,0.0,ModelValue_5,g1,Values[g1]
mu2,fixed,,3.000000e-02,,,3.000000e-02,0.0,ModelValue_6,mu2,Values[mu2]
c,fixed,,2.000000e-02,,,2.000000e-02,0.0,ModelValue_7,c,Values[c]
p2,fixed,,5.000000e+00,,,5.000000e+00,0.0,ModelValue_8,p2,Values[p2]


In [13]:
get_species(model=m)

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id,transition_time,display_name
name,,,,,,,,,,,,,,,
T,compartment,reactions,mmol/ml,1.0,6.022141e+20,,,1.0,6.022141e+20,0.179997,1.083967e+20,Metabolite_0,T,5.555556,T
E,compartment,reactions,mmol/ml,1.0,6.022141e+20,,,1.0,6.022141e+20,-0.010000,-6.022137e+18,Metabolite_1,E,33.333333,E
I,compartment,reactions,mmol/ml,1.0,6.022141e+20,,,1.0,6.022141e+20,-9.999512,-6.021847e+21,Metabolite_2,I,0.100000,I


In [1]:
!rm ../tests/fixtures/smoldyn/caravagna_out.txt
!rm ../tests/fixtures/smoldyn/caravagna.simularium

In [2]:
import smoldyn

smoldyn_sim = smoldyn.Simulation.fromFile('../tests/fixtures/smoldyn/caravagna.txt')

In [ ]:
smoldyn_sim.runSim()

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.smoldyn import SmoldynConverter, SmoldynData
from simulariumio import (
    UnitData, 
    MetaData, 
    DisplayData, 
    DISPLAY_TYPE, 
    ModelMetaData, 
    BinaryWriter, 
    InputFileData,
)
from simulariumio.filters import TranslateFilter


box_size = 50.0


trajectory_data = SmoldynData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="LIP",
            version="8.1",
            authors="Alexander A. Patrie",
            description=(
                "Based on BIOMD0000000498 - Mitchell, Mendes PLoS Comput. Biol. 2013 Nov; 9(11): e1003299: A computational model of liver iron metabolism"
            ),
            doi="10.1371/journal.pcbi.1003299",
            source_code_url="https://github.com/vivarium-collective/bio-bundles",
            source_code_license_url="https://github.com/vivarium-collective/bio-bundles/blob/main/LICENSE",
        ),
    ),
    smoldyn_file=InputFileData(
        file_path="/Users/alexanderpatrie/Desktop/repos/bio-bundles/tests/fixtures/smoldyn/caravagna_out.txt"
    ),
    # display_data=display_data,
    time_units=UnitData("s"),
    spatial_units=UnitData("µm"),  
    center=True,
)


converter = SmoldynConverter(trajectory_data)

simularium_fp = f"../tests/fixtures/smoldyn/caravagna"
converter.add_number_of_agents_plot()
converter.save(simularium_fp, binary=True)

Reading Smoldyn Data -------------
Reading Scatter Plot Data -------------
Converting Trajectory Data to Binary -------------
Writing Binary -------------
saved to ../tests/fixtures/smoldyn/caravagna.simularium
